# 동화 텍스트 생성 AI

## 검증 -> 통합 프롬프트 재구성 -> LLM 생성 로직 구현

1단계: 라이브러리 설치 및 설정

In [1]:
# 1. 라이브러리 설치
!pip install openai -q

import os
from openai import OpenAI
import json
from getpass import getpass

# 2. API Key 입력 (실행 시 입력창에 sk-로 시작하는 키를 넣고 엔터를 치세요)
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key를 입력하세요: ")

client = OpenAI()

print("✅ 설정이 완료되었습니다.")

OpenAI API Key를 입력하세요: ··········
✅ 설정이 완료되었습니다.


2단계: 동화 생성 엔진 (아키텍처 반영 로직)

In [2]:
def generate_fairy_tale(user_input):
    print(f"\n🚀 [프로세스 시작] 사용자 입력: \"{user_input}\"")

    # ------------------------------------------------------------------
    # Step 1: 입력값 검증 및 분석 (Safety & Requirement Check)
    # ------------------------------------------------------------------
    print("1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...")

    check_prompt = f"""
    당신은 동화 서비스의 '안전 관리자'입니다.
    사용자의 입력이 동화를 만들기에 적절한지 판단하세요.
    폭력적, 선정적, 혐오 표현이 있거나 동화와 전혀 관련 없는 요청이라면 'FALSE'를,
    적절하다면 'TRUE'를 출력하세요.
    설명 없이 오직 TRUE 또는 FALSE만 대답하세요.

    사용자 입력: {user_input}
    """

    check_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a safety moderator."},
                  {"role": "user", "content": check_prompt}],
        temperature=0.0
    )

    is_safe = check_response.choices[0].message.content.strip()

    if "FALSE" in is_safe:
        return "⚠️ [차단됨] 부적절하거나 동화 생성과 관련 없는 요청입니다. 다시 입력해주세요."

    print("   -> ✅ 검증 통과.")


    # ------------------------------------------------------------------
    # Step 2: 통합 프롬프트 재구성 (Prompt Re-engineering)
    # 사용자의 짧은 입력을 상세한 '동화 기획서'로 변환합니다.
    # ------------------------------------------------------------------
    print("2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...")

    architect_prompt = f"""
    당신은 전문 '동화 기획자'입니다.
    사용자의 입력을 분석하여 동화 작가가 글을 쓸 수 있도록 상세한 [동화 설계도]를 작성해야 합니다.
    사용자가 구체적으로 말하지 않은 부분은 문맥에 맞게 가장 매력적인 설정으로 채워넣으세요.

    [필수 분석 항목]
    1. 주인공 이름 및 특징 (없으면 어울리는 이름 생성)
    2. 핵심 주제 및 교훈
    3. 배경 (장소, 시간)
    4. 분위기 (예: 몽환적인, 신나는, 차분한 파스텔톤의)
    5. 포함된 문화 요소 (한국 전래동화 풍, 서양 판타지 풍 등)
    6. 줄거리 개요 (기승전결)

    [사용자 입력]
    {user_input}

    [출력 포맷 - JSON 형식만 출력할 것]
    {{
      "title": "동화 제목",
      "characters": "주인공과 등장인물 상세 설정",
      "setting": "배경 설정",
      "tone": "분위기",
      "moral": "교훈",
      "plot_outline": "줄거리 요약"
    }}
    """

    plan_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a creative story architect. Output JSON only."},
                  {"role": "user", "content": architect_prompt}],
        response_format={ "type": "json_object" },
        temperature=0.7
    )

    story_plan = json.loads(plan_response.choices[0].message.content)

    print("\n   📄 [생성된 동화 설계도]")
    print(f"   - 제목: {story_plan['title']}")
    print(f"   - 등장인물: {story_plan['characters']}")
    print(f"   - 교훈: {story_plan['moral']}")
    print(f"   - 분위기: {story_plan['tone']}")


    # ------------------------------------------------------------------
    # Step 3: 동화 텍스트 생성 (LLM Generation)
    # ------------------------------------------------------------------
    print("\n3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...")

    writer_prompt = f"""
    당신은 따뜻하고 교훈적인 이야기를 만드는 베스트셀러 동화 작가입니다.
    아래의 [동화 설계도]를 바탕으로 아이들이 읽기 좋은 동화를 작성해 주세요.

    [작성 가이드]
    1. 문체: '~했어요', '~였답니다'와 같은 부드러운 구어체(경어)를 사용하세요.
    2. 구성: 도입, 전개, 위기, 절정, 결말의 구조를 갖추세요.
    3. 분량: 읽어주었을 때 3분~5분 정도의 길이 (약 1500자 내외).
    4. 강조: 아이의 상상력을 자극하는 의성어, 의태어를 적절히 섞어주세요.

    [동화 설계도]
    {json.dumps(story_plan, ensure_ascii=False)}
    """

    story_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "system", "content": "You are a warm and creative fairy tale writer."},
                  {"role": "user", "content": writer_prompt}],
        temperature=0.8
    )

    final_story = story_response.choices[0].message.content

    return final_story, story_plan

# 함수 테스트
print("✅ 동화 생성 엔진 준비 완료")

✅ 동화 생성 엔진 준비 완료


3단계: 실행 및 테스트

In [3]:
# === 사용자 입력 테스트 ===
user_prompt = input("동화로 만들고 싶은 내용을 입력하세요 (예: 잠자기 싫어하는 아기 호랑이 이야기, 친구와 싸워서 속상한 민수 이야기 등): ")

if user_prompt:
    story_text, story_metadata = generate_fairy_tale(user_prompt)

    print("\n" + "="*50)
    print(f"📚 제목: {story_metadata['title']}")
    print("="*50)
    print(story_text)
    print("="*50)
else:
    print("입력값이 없습니다.")

동화로 만들고 싶은 내용을 입력하세요 (예: 잠자기 싫어하는 아기 호랑이 이야기, 친구와 싸워서 속상한 민수 이야기 등): 잠자기 싫어하는 아기 호랑이 이야기

🚀 [프로세스 시작] 사용자 입력: "잠자기 싫어하는 아기 호랑이 이야기"
1️⃣ [검증 단계] 입력값의 안전성과 적합성을 판단 중...
   -> ✅ 검증 통과.
2️⃣ [기획 단계] 사용자 의도를 분석하여 '통합 프롬프트' 생성 중...

   📄 [생성된 동화 설계도]
   - 제목: 잠들기 싫은 아기 호랑이의 모험
   - 등장인물: {'main_character': {'name': '호야', 'description': '잠들기 싫어하고 에너지가 넘치는 호기심 많은 아기 호랑이'}, 'supporting_characters': [{'name': '바람 도깨비', 'description': '밤의 바람을 타고 오는 장난꾸러기 도깨비로 호야에게 잠을 자라고 조언하는 존재'}, {'name': '달빛 요정', 'description': '달빛을 가르치며 호야에게 꿈의 세계를 보여주는 따뜻한 성격의 요정'}]}
   - 교훈: 휴식과 잠은 더 큰 모험과 성장을 위한 중요한 준비 단계임을 깨닫는다.
   - 분위기: 몽환적이고 따뜻한

3️⃣ [생성 단계] 설계도를 바탕으로 GPT-4o가 동화를 집필 중입니다...

📚 제목: 잠들기 싫은 아기 호랑이의 모험
옛날 옛적, 깊고 푸른 산 속에 에너지가 넘치는 아기 호랑이 호야가 살고 있었어요. 호야는 밤이 되어도 잠들기를 싫어했어요. "잠들기에는 너무 재미있는 일이 많아!" 호야는 퐁퐁 뛰며 말하곤 했답니다. 별이 반짝이는 밤, 호야의 호기심은 더욱 커져갔어요.

하루는 호야가 잠을 거부하고 산 속을 탐험하기로 했어요. "누가 나를 멈출 수 있을까?" 라고 중얼거리며 호야는 깊숙한 숲으로 들어갔지요. 바람이 살랑살랑 불고, 나뭇잎들이 사각사각 춤을 추고 있었어요. 그때, "휘익~ 휘익~" 하는 소리가 들렸어요. 그것은 바로 바람 도깨비였답니다. 바람 도깨비